In [1]:
# IRISデータセットの読み込み

from sklearn.datasets import load_iris
import pandas as pd
import numpy as np
iris = load_iris()

iris_data = pd.DataFrame(data=iris.data, columns=iris.feature_names)
iris_label = pd.Series(data=iris.target)

X = iris_data.iloc[0:100, [0,2]].values 
y_tmp = iris_label.iloc[0:100,].values
y = np.where(y_tmp == 0, -1 , 1)

In [2]:
# 標準化
X_std = np.copy(X)
X_std[:, 0] = (X[:, 0] - X[:, 0].mean()) / X[:, 0].std()
X_std[:, 1] = (X[:, 1] - X[:, 1].mean()) / X[:, 1].std()

In [14]:
class AdalineSGD(object):
    def __init__(self, eta=0.01, n_iter=10, shuffle=True, random_state=None):
        self.eta = eta
        self.n_iter = n_iter
        self.w_initialized = False
        self.shuffle = shuffle
        self.random_state = random_state
        
    def fit(self, X, y):
        # 初期値を設定
        self._initialize_weights(X.shape[1])
        self.cost_ = []
        for i in range(self.n_iter):
            # shuffle変数がTrueなら、X・yのインデックスをシャッフルしてランダムな順序に並べ替える
            if self.shuffle:
                X, y = self._shuffle(X, y)
            cost = []
            for xi, target in zip(X, y):
                cost.append(self._update_weights(xi, target))
            # コスト関数の総和を目的変数の個数で割る
            avg_cost = sum(cost) / len(y)
            self.cost_.append(avg_cost)
        return self

    # オンライン学習に合わせて重みの再初期化を行わないメソッド
    def partial_fit(self, X, y):
        if not self.w_initialized:
            self._initialize_weights(X.shape[1])
        # shape[0] で1つめの配列の長さを取得
        if y.ravel().shape[0] > 1:
            # 各行ごとに処理
            for xi, target in zip(X, y):
                self._update_weights(xi, target)
        else:
            # 1行だけならforで回す必要はないよね
            self._update_weights(X, y)
        return self

    def _shuffle(self, X, y):
        r = self.rgen.permutation(len(y))
        return X[r], y[r]
    
    def _initialize_weights(self, m):
        self.rgen = np.random.RandomState(self.random_state)
        self.w_ = self.rgen.normal(loc=0.0, scale=0.01, size=1 + m)
        self.w_initialized = True
        
    def _update_weights(self, xi, target):
        # 総出力を計算
        output = self.activation(self.net_input(xi))
        # 目的変数に対する各行の総出力の誤差を計算
        error = (target - output)
        # 学習率 × (Xの各列 × 各行の出力誤差)
        self.w_[1:] += self.eta * xi.dot(error)
        # 学習率 × 各行の出力誤差の総和
        self.w_[0] += self.eta * error
        # コスト関数の計算(平均二乗誤差 * 1/2)
        cost = 0.5 * error**2
        return cost
    
    def net_input(self, X):
        return np.dot(X, self.w_[1:]) + self.w_[0]

    def activation(self, X):
        return X

    def predict(self, X):
        return np.where(self.activation(self.net_input(X)) >= 0.0, 1, -1)

In [15]:
ada = AdalineSGD(n_iter=15, eta=0.01, random_state=1)
#ada.fit(X_std, y)
ada.partial_fit(X_std, y)

[-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1]
100
100
